In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

configs = [
    dict(
        symbol='NDX', timeframe='H1', direction='buy', pruner_type='successive',
        train_start=datetime(2020,4,1), train_end=datetime(2025,4,1),
        test_start=datetime(2022,4,1),  test_end=datetime(2023,4,1),
        label_method='random', search_type='reliability', search_subtype='',
        n_models=1, n_jobs=-1, debug=False,
    ),
]

# Crear tag para cada configuración
for cfg in configs:
    # Construir el tag asegurando que no haya dobles guiones bajos por campos vacíos
    tag_parts = [
        cfg['symbol'],
        cfg['timeframe'],
        cfg['direction'],
        cfg['label_method'][:2],
        cfg['search_type'][:3],
        (cfg.get('search_subtype') or '')[:2],
    ]
    # Filtrar partes vacías y unir con "_"
    cfg["tag"] = "_".join([part for part in tag_parts if part]).strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None

n_trials = 5000
if study:
    n_trials = len(study.trials) + n_trials

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[NDX_H1_buy_ra_rel] modelo 0 trial 11/5010 score=0.458172 avg=16.90s mem=964.39MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 12/5010 score=0.458172 avg=26.25s mem=1415.10MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 13/5010 score=0.458172 avg=28.62s mem=1401.92MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 14/5010 score=0.534492 avg=29.89s mem=1398.38MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 15/5010 score=0.534492 avg=44.57s mem=1838.12MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 16/5010 score=0.534492 avg=52.49s mem=1876.80MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 17/5010 score=0.534492 avg=49.46s mem=1876.80MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 18/5010 score=0.534492 avg=48.68s mem=1864.50MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 19/5010 score=0.534492 avg=48.55s mem=1879.47MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 20/5010 score=0.534492 avg=49.14s mem=1890.10MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 21/5010 score=0.534492 avg=49.45s mem=1892.78MB 
[NDX_H1_buy_ra_rel] modelo 0 trial 22/5010 score=0.534492 avg=50.14s mem=1918